1. Get a training corpus
2. Tokenize the corpus and set a vocabulary for an embedding model
3. Train the embedding model
4. Set a classification model
5. Use the trained embedding model for the classification task

In [ ]:
# numpy를 호환 가능한 버전으로 강제로 재설치
!pip install numpy==1.23.5 --force-reinstall

# gensim도 재설치
!pip install gensim --force-reinstall

  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
bigframes 1.42.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have n

  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
^C


In [ ]:
from gensim.models import FastText
from gensim.utils import simple_preprocess

import numpy as np
from collections import Counter

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

from sklearn.metrics import f1_score

## 1. Get a training corpus

movie review corpus (MR).



In [ ]:
# download the file
!wget https://github.com/FKarl/short-text-classification/raw/refs/heads/main/data/corpus/mr_shuffle.txt

--2025-04-14 19:27:18--  https://github.com/FKarl/short-text-classification/raw/refs/heads/main/data/corpus/mr_shuffle.txt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FKarl/short-text-classification/refs/heads/main/data/corpus/mr_shuffle.txt [following]
--2025-04-14 19:27:18--  https://raw.githubusercontent.com/FKarl/short-text-classification/refs/heads/main/data/corpus/mr_shuffle.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1200403 (1.1M) [text/plain]
Saving to: ‘mr_shuffle.txt.1’

mr_shuffle.txt.1    100%[===================>]   1.14M  4.20MB/s    in 0.3s    

2025-04-14 19:27:19 

In [ ]:
# we have the file in current location: mr_shuffle.txt
!ls -alh

total 780M
drwxr-xr-x 1 root root 4.0K Apr 14 19:27 .
drwxr-xr-x 1 root root 4.0K Apr 14 10:44 ..
drwxr-xr-x 4 root root 4.0K Apr 10 13:37 .config
-rw-r--r-- 1 root root 7.9M Apr 14 19:16 core_emb
-rw-r--r-- 1 root root 763M Apr 14 19:16 core_emb.wv.vectors_ngrams.npy
drwxr-xr-x 2 root root 4.0K Apr 14 18:03 .ipynb_checkpoints
-rw-r--r-- 1 root root 1.2M Apr 14 17:48 mr_shuffle.txt
-rw-r--r-- 1 root root 1.2M Apr 14 19:27 mr_shuffle.txt.1
drwxr-xr-x 1 root root 4.0K Apr 10 13:37 sample_data
-rw-r--r-- 1 root root 5.4M Apr 14 18:40 sst2_model.pt
-rw-r--r-- 1 root root 185K Apr 14 17:02 sst2-test.txt
-rw-r--r-- 1 root root 704K Apr 14 16:58 sst2-train.txt


In [ ]:
# check the first 10 lines of the mr_shuffle.txt file
!head mr_shuffle.txt

uncertain in tone a garbled exercise in sexual politics , a junior varsity short cuts by way of very bad things
weighted down with slow , uninvolving storytelling and flat acting
the tale of her passionate , tumultuous affair with musset unfolds as sand 's masculine persona , with its love of life and beauty , takes form
anyone who suffers through this film deserves , at the very least , a big box of consolation candy
it 's a gag that 's worn a bit thin over the years , though do n't ask still finds a few chuckles
paid in full is remarkably engaging despite being noticeably derivative of goodfellas and at least a half dozen other trouble in the ghetto flicks
you have no affinity for most of the characters nothing about them is attractive what they see in each other also is difficult to fathom
the best thing i can say about this film is that i ca n't wait to see what the director does next
when you think you 've figured out bielinsky 's great game , that 's when you 're in the most trou

In [ ]:
# check the size of the test.tsv file: it has 10,661 lines
!wc mr_shuffle.txt

  10661  217354 1200403 mr_shuffle.txt


## 2. Tokenize the corpus and set a vocabulary for an embedding model

In [ ]:
# text processing
sentences = []
with open('mr_shuffle.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if line:
            tokens = simple_preprocess(line)
            sentences.append(tokens)

 # FastText는 단어를 내부적으로 subword 단위로 쪼개서 학습하므로 외부에서 굳이 단어보다 더 작게 자를 필요가 없어서
 # 간단한 단어 수준의 tokenization 방식인 simple_preprocess을 채택했습니다.

##3. train the embedding model

In [ ]:
# FastText 선택 이유: 영화 리뷰에는 구어체에서 쓰이는 희귀한 단어, 오타, 비표준어 등이 많을 것으로 추측되므로,
# 단어를 subword로 쪼개서 학습하여 OOV에도 어느 정도 의미있는 벡터를 제공하는 FastText가 적합할 것이라고 판단했습니다.

embedding_model = FastText(
    sentences=sentences,
    vector_size=100, # 각 단어 벡터 차원 수
    window=5, # 앞뒤로 5단어까지 문맥으로 간주
    min_count=2, # 최소 등장 횟수가 2번 이상인 단어만 학습
    workers=4, # 학습에 사용할 쓰레드 수
    sg=1, # Skip-gram 모델 사용 (희귀한 단어를 더 잘 다루고자 선택했습니다)
    epochs=10 # 학습 반복 횟수
)

In [ ]:
# save the trained embedding model
embedding_model.save("core_emb")

## 4. Set a classification model

classification pipeline using a simple Logistic Regression model.

* Task: SST2 (Stanford Sentiment Treebank; Binary Classification)
* Text: movie review for sentiment classification
* Label: 1 (Positive), 0 (Negative)

In [ ]:
# training set

!wget https://github.com/FKarl/short-text-classification/raw/refs/heads/main/data/sst2/sst2-train.txt

--2025-04-14 19:28:12--  https://github.com/FKarl/short-text-classification/raw/refs/heads/main/data/sst2/sst2-train.txt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FKarl/short-text-classification/refs/heads/main/data/sst2/sst2-train.txt [following]
--2025-04-14 19:28:13--  https://raw.githubusercontent.com/FKarl/short-text-classification/refs/heads/main/data/sst2/sst2-train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 720259 (703K) [text/plain]
Saving to: ‘sst2-train.txt.1’

sst2-train.txt.1    100%[===================>] 703.38K  2.77MB/s    in 0.2s    

2025-04-14 19:28:14 (2.77 M

In [ ]:
# test set

!wget https://github.com/FKarl/short-text-classification/raw/refs/heads/main/data/sst2/sst2-test.txt

--2025-04-14 19:28:17--  https://github.com/FKarl/short-text-classification/raw/refs/heads/main/data/sst2/sst2-test.txt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FKarl/short-text-classification/refs/heads/main/data/sst2/sst2-test.txt [following]
--2025-04-14 19:28:18--  https://raw.githubusercontent.com/FKarl/short-text-classification/refs/heads/main/data/sst2/sst2-test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 188749 (184K) [text/plain]
Saving to: ‘sst2-test.txt.1’

sst2-test.txt.1     100%[===================>] 184.33K  1.16MB/s    in 0.2s    

2025-04-14 19:28:18 (1.16 MB/s)

two columns in each dataset file:
* label: 1 (positive), 0 (negative)
* movie review text

In [ ]:
# check the sample lines and columns

!head sst2-train.txt

1	a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films
0	apparently reassembled from the cutting room floor of any given daytime soap
0	they presume their audience wo n't sit still for a sociology lesson , however entertainingly presented , so they trot out the conventional science fiction elements of bug eyed monsters and futuristic women in skimpy clothes
1	this is a visually stunning rumination on love , memory , history and the war between art and commerce
1	jonathan parker 's bartleby should have been the be all end all of the modern office anomie films
1	campanella gets the tone just right funny in the middle of sad in the middle of hopeful
0	a fan film that for the uninitiated plays better on video with the sound turned down
1	b art and berling are both superb , while huppert is magnificent
0	a little less extreme than in the past , with longer exposition sequences between them , and with fewer gags to break the tedium
0	the fil

In [ ]:
# check the data sizes

!wc sst2-train.txt # 6919 lines
!wc sst2-test.txt # 1820 lines

  6919 135016 720259 sst2-train.txt
  1820  35474 188749 sst2-test.txt


In [ ]:
# convert texts to sequences

sentences = []
y_train = []

# training data 불러오기
with open('sst2-train.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        label, text = line.split('\t', 1)
        y_train.append(int(label))
        sentences.append(text)

# tokenization
tokenized_sentences = [sent.split() for sent in sentences]

# 등장 빈도순으로 정렬
word_list = []
for sent in tokenized_sentences:
    for word in sent:
      word_list.append(word)

word_counts = Counter(word_list)

# vocab 생성
vocab = sorted(word_counts, key=word_counts.get, reverse=True)

word_to_index = {}
word_to_index['<PAD>'] = 0
word_to_index['<UNK>'] = 1

for index, word in enumerate(vocab) :
  word_to_index[word] = index + 2

vocab_size = len(word_to_index)

# 텍스트를 시퀀스로 변환
def texts_to_sequences(tokenized_X_data, word_to_index):
  encoded_X_data = []
  for sent in tokenized_X_data:
    index_sequences = []
    for word in sent:
      try:
          index_sequences.append(word_to_index[word])
      except KeyError:
          index_sequences.append(word_to_index['<UNK>']) # vocabulary에 없는 단어라면 unknown에 해당하는 수로 바꾸기.
    encoded_X_data.append(index_sequences)
  return encoded_X_data

X_encoded = texts_to_sequences(tokenized_sentences, word_to_index)
max_len = max(len(l) for l in X_encoded)

# 시퀀스 padding
def pad_sequences(sentences, max_len):
  features = np.zeros((len(sentences), max_len), dtype=int)
  for index, sentence in enumerate(sentences):
    if len(sentence) != 0:
      features[index, :len(sentence)] = np.array(sentence)[:max_len]
  return features

X_train = pad_sequences(X_encoded, max_len=max_len)
y_train = np.array(y_train)

In [ ]:
# define the model class
# define the 'Logistic Regression' model you learned!

# 앞서 학습시킨 FastText 임베딩 로드
ft_model = FastText.load("core_emb")

# 2차원 임베딩 매트릭스 생성
embedding_matrix = np.zeros((len(word_to_index), ft_model.vector_size))

for word, idx in word_to_index.items():
    if word in ft_model.wv:
        embedding_matrix[idx] = ft_model.wv[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(ft_model.vector_size,))  # OOV


class SimpleModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SimpleModel, self).__init__()
        # embedding layer에 앞서 만든 임베딩 적용.
        # freeze=False : 임베딩 파인튜닝 허용. 사전 학습된 임베딩을 학습 중에도 업데이트할 수 있게 함.
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=False)
         # linear layer는 1차원을 받으므로 2차원 embedding을 1차원으로 펼침.
        self.flatten = nn.Flatten()
        # linear layer 1개. 문장이 들어왔을 때 positive한 layer로 나올 확률.
        self.fc = nn.Linear(embedding_dim * max_len, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # embedded.shape == (배치 크기, 문장의 길이, 임베딩 벡터의 차원)
        embedded = self.embedding(x)

        # flattend.shape == (배치 크기, 문장의 길이 × 임베딩 벡터의 차원)
        flattened = self.flatten(embedded)

        # output.shape == (배치 크기, 1)
        output = self.fc(flattened)
        return self.sigmoid(output)

In [ ]:
# instantiate the model, criterion(loss), and optimizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

embedding_dim = 100
simple_model = SimpleModel(vocab_size, embedding_dim).to(device)

# nn에서 loss를 골라와서 세팅해준다.
# BCELoss는 이진 분류 문제에 적합한 loss function이고 sigmoid와 함께 사용되어 선택했습니다.
criterion = nn.BCELoss()

# parameter를 optimizer에 넣어준다.
# Adam은 별다른 튜닝 없이도 좋은 성능을 내는 대표적인 최적화 알고리즘이라 선택했습니다.
optimizer = Adam(simple_model.parameters())

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.long), torch.tensor(y_train, dtype=torch.float32))
train_dataloader = DataLoader(train_dataset, batch_size=2) # batch 단위로 묶어준다. 여기에서는 데이터 2개를 한 묶음으로 만들어 처리한다.

##5. use the trained embedding model for the classification task

In [ ]:
# train the model!
# train the model for 5 epochs
# and print the training loss at the end of each epoch

for epoch in range(5):
    # input: index로 되어있는 문장. for문이 2번이므로 training data 내 모든 batch에 대해 반복한다.
    for inputs, targets in train_dataloader:
        # inputs.shape == (배치 크기, 문장 길이)
        # targets.shape == (배치 크기)
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        # outputs.shape == (배치 크기)
        outputs = simple_model(inputs).view(-1) # 이 자체가 확률이 된다.

        loss = criterion(outputs, targets)
        loss.backward()

        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.2230575829744339
Epoch 2, Loss: 0.14312806725502014
Epoch 3, Loss: 0.06967110931873322
Epoch 4, Loss: 0.006739622447639704
Epoch 5, Loss: 0.0008573121158406138


In [ ]:
# save the trained model
# filename: 'sst2_model.pt'
torch.save(simple_model.state_dict(), 'sst2_model.pt')

* Use `f1 score` for the evaluation metric.

  * See also: https://scikit-learn.org/0.15/modules/generated/sklearn.metrics.f1_score.html

  * Use 'macro average'!
  * The output will be in the form of a single scalar (numerical) value.

In [ ]:
# convert test texts to sequences

x_test = []
y_test = []

with open('sst2-test.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        label, text = line.split('\t', 1)
        y_test.append(int(label))
        x_test.append(text)

x_test = [i.split() for i in x_test] # test data를 tokenize한다.
x_test = texts_to_sequences(x_test, word_to_index) # index로 바꿔준다. vocabulary에 없는 것은 unknown으로 처리된다.
x_test = pad_sequences(x_test, max_len=max_len)
y_test = np.array(y_test)

test_dataset = TensorDataset(torch.tensor(x_test, dtype=torch.long), torch.tensor(y_test, dtype=torch.float32))
test_dataloader = DataLoader(test_dataset, batch_size=2)

In [ ]:
# test the model
# print the f1 score for the test dataset
y_true = []
y_pred = []

# 모델을 evaluation mode로 바꿔준다. 성능을 테스트할 것이므로 더이상 gradient update을 하지 않겠다는 뜻이다.
simple_model.eval()

for inputs, targets in test_dataloader:
    # inputs.shape == (배치 크기, 문장 길이)
    # targets.shape == (배치 크기)
    inputs, targets = inputs.to(device), targets.to(device)

    # outputs.shape == (배치 크기)
    outputs = simple_model(inputs).view(-1)
    preds = (outputs > 0.5).long().view(-1)

    y_pred.extend(preds.tolist())
    y_true.extend(targets.tolist())


# test dataset에 대한 f1 score
print(f1_score(y_true, y_pred, average='macro'))

0.758868892727763
